In [59]:
# import libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os
import numpy as np
import re
import random
import nltk
from scipy import sparse
from scipy.sparse import csr_matrix, vstack
from textblob import TextBlob
from langdetect import detect_langs
import pickle
from datetime import datetime

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [60]:
result = pd.read_csv('./fakejob_balance.csv')
result

,has_company_logo,n_stop_words,p_noun,p_adj,p_verb,p_adv,n_stems,n_unique_stems,unique_stems_ratio,p_stop_words,p_noun_words,p_adj_words,p_verb_words,p_adv_words,tf_idf_score,polarity,subjectivity,artist
0,1,151,0.393120,0.093366,0.157248,0.014742,407,228,0.560197,0.371007,0.393120,0.093366,0.157248,0.014742,11.005919,0.157504,0.445126,0
1,1,148,0.420366,0.083551,0.135770,0.007833,383,189,0.493473,0.386423,0.420366,0.083551,0.135770,0.007833,10.861920,0.289571,0.621981,0
2,1,247,0.419137,0.099730,0.137466,0.018868,742,352,0.474394,0.332884,0.419137,0.099730,0.137466,0.018868,14.254465,0.214685,0.459548,0
3,1,134,0.529880,0.099602,0.109562,0.017928,502,292,0.581673,0.266932,0.529880,0.099602,0.109562,0.017928,13.966813,0.164791,0.298529,0
4,0,172,0.395626,0.123260,0.125249,0.015905,503,265,0.526839,0.341948,0.395626,0.123260,0.125249,0.015905,12.454960,0.177687,0.492891,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16992,1,78,0.438462,0.092308,0.119231,0.026923,260,162,0.623077,0.300000,0.438462,0.092308,0.119231,0.026923,10.048639,0.097737,0.275495,0
16993,0,10,0.676471,0.044118,0.073529,0.014706,68,59,0.867647,0.147059,0.676471,0.044118,0.073529,0.014706,6.326387,0.128939,0.417576,0
16994,1,253,0.414773,0.093750,0.129261,0.032670,704,371,0.526989,0.359375,0.414773,0.093750,0.129261,0.032670,13.083046,0.118523,0.474795,0
16995,0,127,0.458445,0.096515,0.104558,0.026810,373,198,0.530831,0.340483,0.458445,0.096515,0.104558,0.026810,11.065834,0.186843,0.434816,0


In [61]:
feature = result.drop(columns='artist')
target = result.artist


In [62]:
feature

,has_company_logo,n_stop_words,p_noun,p_adj,p_verb,p_adv,n_stems,n_unique_stems,unique_stems_ratio,p_stop_words,p_noun_words,p_adj_words,p_verb_words,p_adv_words,tf_idf_score,polarity,subjectivity
0,1,151,0.393120,0.093366,0.157248,0.014742,407,228,0.560197,0.371007,0.393120,0.093366,0.157248,0.014742,11.005919,0.157504,0.445126
1,1,148,0.420366,0.083551,0.135770,0.007833,383,189,0.493473,0.386423,0.420366,0.083551,0.135770,0.007833,10.861920,0.289571,0.621981
2,1,247,0.419137,0.099730,0.137466,0.018868,742,352,0.474394,0.332884,0.419137,0.099730,0.137466,0.018868,14.254465,0.214685,0.459548
3,1,134,0.529880,0.099602,0.109562,0.017928,502,292,0.581673,0.266932,0.529880,0.099602,0.109562,0.017928,13.966813,0.164791,0.298529
4,0,172,0.395626,0.123260,0.125249,0.015905,503,265,0.526839,0.341948,0.395626,0.123260,0.125249,0.015905,12.454960,0.177687,0.492891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16992,1,78,0.438462,0.092308,0.119231,0.026923,260,162,0.623077,0.300000,0.438462,0.092308,0.119231,0.026923,10.048639,0.097737,0.275495
16993,0,10,0.676471,0.044118,0.073529,0.014706,68,59,0.867647,0.147059,0.676471,0.044118,0.073529,0.014706,6.326387,0.128939,0.417576
16994,1,253,0.414773,0.093750,0.129261,0.032670,704,371,0.526989,0.359375,0.414773,0.093750,0.129261,0.032670,13.083046,0.118523,0.474795
16995,0,127,0.458445,0.096515,0.104558,0.026810,373,198,0.530831,0.340483,0.458445,0.096515,0.104558,0.026810,11.065834,0.186843,0.434816


In [63]:
result

,has_company_logo,n_stop_words,p_noun,p_adj,p_verb,p_adv,n_stems,n_unique_stems,unique_stems_ratio,p_stop_words,p_noun_words,p_adj_words,p_verb_words,p_adv_words,tf_idf_score,polarity,subjectivity,artist
0,1,151,0.393120,0.093366,0.157248,0.014742,407,228,0.560197,0.371007,0.393120,0.093366,0.157248,0.014742,11.005919,0.157504,0.445126,0
1,1,148,0.420366,0.083551,0.135770,0.007833,383,189,0.493473,0.386423,0.420366,0.083551,0.135770,0.007833,10.861920,0.289571,0.621981,0
2,1,247,0.419137,0.099730,0.137466,0.018868,742,352,0.474394,0.332884,0.419137,0.099730,0.137466,0.018868,14.254465,0.214685,0.459548,0
3,1,134,0.529880,0.099602,0.109562,0.017928,502,292,0.581673,0.266932,0.529880,0.099602,0.109562,0.017928,13.966813,0.164791,0.298529,0
4,0,172,0.395626,0.123260,0.125249,0.015905,503,265,0.526839,0.341948,0.395626,0.123260,0.125249,0.015905,12.454960,0.177687,0.492891,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16992,1,78,0.438462,0.092308,0.119231,0.026923,260,162,0.623077,0.300000,0.438462,0.092308,0.119231,0.026923,10.048639,0.097737,0.275495,0
16993,0,10,0.676471,0.044118,0.073529,0.014706,68,59,0.867647,0.147059,0.676471,0.044118,0.073529,0.014706,6.326387,0.128939,0.417576,0
16994,1,253,0.414773,0.093750,0.129261,0.032670,704,371,0.526989,0.359375,0.414773,0.093750,0.129261,0.032670,13.083046,0.118523,0.474795,0
16995,0,127,0.458445,0.096515,0.104558,0.026810,373,198,0.530831,0.340483,0.458445,0.096515,0.104558,0.026810,11.065834,0.186843,0.434816,0


In [64]:
from sklearn.neighbors import KNeighborsClassifier

In [65]:
import random
knn = KNeighborsClassifier()
knn.fit(feature.values,target.values)
neighborpoint = knn.kneighbors(feature.values, 5, False)

for i in range(len(feature)):
    if(result['artist'].iloc[i]==1):
        for j in neighborpoint[i]:
            if(result['artist'].iloc[j]==1):
                for k in range(8):
                    has_company_logo = feature['has_company_logo'].iloc[i] + (feature['has_company_logo'].iloc[j] - feature['has_company_logo'].iloc[i])*random.random()
                    has_company_logo = 1 if has_company_logo>=0.5 else 0
                    n_stop_words = int(feature['n_stop_words'].iloc[i] + (feature['n_stop_words'].iloc[j] - feature['n_stop_words'].iloc[i])*random.random())
                    p_noun = feature['p_noun'].iloc[i] + (feature['p_noun'].iloc[j] - feature['p_noun'].iloc[i])*random.random()
                    p_adj = feature['p_adj'].iloc[i] + (feature['p_adj'].iloc[j] - feature['p_adj'].iloc[i])*random.random()
                    p_verb = feature['p_verb'].iloc[i] + (feature['p_verb'].iloc[j] - feature['p_verb'].iloc[i])*random.random()
                    p_adv = feature['p_adv'].iloc[i] + (feature['p_adv'].iloc[j] - feature['p_adv'].iloc[i])*random.random()
                    n_stems = int(feature['n_stems'].iloc[i] + (feature['n_stems'].iloc[j] - feature['n_stems'].iloc[i])*random.random())
                    n_unique_stems = int(feature['n_unique_stems'].iloc[i] + (feature['n_unique_stems'].iloc[j] - feature['n_unique_stems'].iloc[i])*random.random())
                    unique_stems_ratio = feature['unique_stems_ratio'].iloc[i] + (feature['unique_stems_ratio'].iloc[j] - feature['unique_stems_ratio'].iloc[i])*random.random()
                    p_stop_words = feature['p_stop_words'].iloc[i] + (feature['p_stop_words'].iloc[j] - feature['p_stop_words'].iloc[i])*random.random()
                    p_noun_words = feature['p_noun_words'].iloc[i] + (feature['p_noun_words'].iloc[j] - feature['p_noun_words'].iloc[i])*random.random()
                    p_adj_words = feature['p_adj_words'].iloc[i] + (feature['p_adj_words'].iloc[j] - feature['p_adj_words'].iloc[i])*random.random()
                    p_verb_words = feature['p_verb_words'].iloc[i] + (feature['p_verb_words'].iloc[j] - feature['p_verb_words'].iloc[i])*random.random()
                    p_adv_words = feature['p_adv_words'].iloc[i] + (feature['p_adv_words'].iloc[j] - feature['p_adv_words'].iloc[i])*random.random()
                    tf_idf_score = feature['tf_idf_score'].iloc[i] + (feature['tf_idf_score'].iloc[j] - feature['tf_idf_score'].iloc[i])*random.random()
                    polarity = feature['polarity'].iloc[i] + (feature['polarity'].iloc[j] - feature['polarity'].iloc[i])*random.random()
                    subjectivity = feature['subjectivity'].iloc[i] + (feature['subjectivity'].iloc[j] - feature['subjectivity'].iloc[i])*random.random()
                    result.loc[len(result.index)] = [has_company_logo, n_stop_words, p_noun, p_adj, p_verb, p_adv, n_stems, n_unique_stems, unique_stems_ratio, p_stop_words, 
                                                    p_noun_words, p_adj_words, p_verb_words, p_adv_words, tf_idf_score, polarity, subjectivity, 1]

In [66]:

result


,has_company_logo,n_stop_words,p_noun,p_adj,p_verb,p_adv,n_stems,n_unique_stems,unique_stems_ratio,p_stop_words,p_noun_words,p_adj_words,p_verb_words,p_adv_words,tf_idf_score,polarity,subjectivity,artist
0,1.0,151.0,0.393120,0.093366,0.157248,0.014742,407.0,228.0,0.560197,0.371007,0.393120,0.093366,0.157248,0.014742,11.005919,0.157504,0.445126,0.0
1,1.0,148.0,0.420366,0.083551,0.135770,0.007833,383.0,189.0,0.493473,0.386423,0.420366,0.083551,0.135770,0.007833,10.861920,0.289571,0.621981,0.0
2,1.0,247.0,0.419137,0.099730,0.137466,0.018868,742.0,352.0,0.474394,0.332884,0.419137,0.099730,0.137466,0.018868,14.254465,0.214685,0.459548,0.0
3,1.0,134.0,0.529880,0.099602,0.109562,0.017928,502.0,292.0,0.581673,0.266932,0.529880,0.099602,0.109562,0.017928,13.966813,0.164791,0.298529,0.0
4,0.0,172.0,0.395626,0.123260,0.125249,0.015905,503.0,265.0,0.526839,0.341948,0.395626,0.123260,0.125249,0.015905,12.454960,0.177687,0.492891,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32808,1.0,57.0,0.419580,0.041958,0.146853,0.020979,143.0,102.0,0.713287,0.398601,0.419580,0.041958,0.146853,0.020979,8.560112,0.290000,0.440000,1.0
32809,1.0,57.0,0.419580,0.041958,0.146853,0.020979,143.0,102.0,0.713287,0.398601,0.419580,0.041958,0.146853,0.020979,8.560112,0.290000,0.440000,1.0
32810,1.0,57.0,0.419580,0.041958,0.146853,0.020979,143.0,102.0,0.713287,0.398601,0.419580,0.041958,0.146853,0.020979,8.560112,0.290000,0.440000,1.0
32811,1.0,57.0,0.419580,0.041958,0.146853,0.020979,143.0,102.0,0.713287,0.398601,0.419580,0.041958,0.146853,0.020979,8.560112,0.290000,0.440000,1.0


In [67]:
print(result[result['artist']==0].shape)
print(result[result['artist']==1].shape)

(16161, 18)
(16652, 18)


In [68]:
feature = result.drop(columns='artist')
target = result.artist

In [69]:
from sklearn.model_selection import train_test_split
train_feature, test_feature,train_category, test_category = train_test_split(feature, target, test_size = 0.2 , random_state = 0)

In [70]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

lg=LogisticRegression(solver='lbfgs', max_iter=3000)
svmc = svm.SVC(decision_function_shape='ovr', probability=True)
rf = RandomForestClassifier(n_estimators=50)
ada = AdaBoostClassifier(n_estimators=100)

lg_model=lg.fit(train_feature.values,train_category.values)
svmc_model = svmc.fit(train_feature.values,train_category.values)
rf_model = rf.fit(train_feature.values,train_category.values)
ada_model = ada.fit(train_feature.values,train_category.values)

In [71]:
lg_predict=lg.predict(test_feature.values)
svm_predict = svmc.predict(test_feature.values)
rf_predict = rf.predict(test_feature.values)
ada_predict = ada.predict(test_feature.values)

lg_score=accuracy_score(test_category.values,lg_predict)
svm_score=accuracy_score(test_category.values, svm_predict)
rf_score=accuracy_score(test_category.values, rf_predict)
ada_score=accuracy_score(test_category.values, ada_predict)
print("logicstic regression score :", lg_score)
print("SVM score :", svm_score)
print("Random Forest score: ", rf_score)
print("Ada Boost score: ", ada_score)

logicstic regression score : 0.7697699222916349
SVM score : 0.7132408959317386
Random Forest score:  0.9904007313728478
Ada Boost score:  0.8439737924729545


In [72]:
from sklearn.metrics import classification_report
report = classification_report(test_category.values, lg_predict, target_names = ['0', '1'])
print(report)
report2 = classification_report(test_category.values, svm_predict, target_names= ['0', '1'])
print(report2)

              precision    recall  f1-score   support

           0       0.75      0.81      0.78      3244
           1       0.80      0.73      0.76      3319

    accuracy                           0.77      6563
   macro avg       0.77      0.77      0.77      6563
weighted avg       0.77      0.77      0.77      6563

              precision    recall  f1-score   support

           0       0.73      0.66      0.70      3244
           1       0.70      0.76      0.73      3319

    accuracy                           0.71      6563
   macro avg       0.71      0.71      0.71      6563
weighted avg       0.71      0.71      0.71      6563



In [73]:
report = classification_report(test_category.values, rf_predict, target_names = ['0', '1'])
print(report)
report2 = classification_report(test_category.values, ada_predict, target_names= ['0', '1'])
print(report2)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      3244
           1       0.99      0.99      0.99      3319

    accuracy                           0.99      6563
   macro avg       0.99      0.99      0.99      6563
weighted avg       0.99      0.99      0.99      6563

              precision    recall  f1-score   support

           0       0.84      0.85      0.84      3244
           1       0.85      0.84      0.84      3319

    accuracy                           0.84      6563
   macro avg       0.84      0.84      0.84      6563
weighted avg       0.84      0.84      0.84      6563

